# DATA

## Getting DATA

In [1]:
# installed accelerate

In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.6 MB/s eta 0:00:00


In [3]:
!pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [5]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [6]:
import nltk

import evaluate
import numpy as np

In [7]:
import torch
torch.cuda.empty_cache()

In [8]:
import pandas as pd
import datasets
from transformers import T5Tokenizer, TFT5ForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AutoModelForCausalLM, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer, TFTrainingArguments

In [9]:
data1 = pd.read_csv("/content/data2.csv")

In [10]:
data1.head(3)

,Unnamed: 0,Repository,Message,Diff
0,0,swaroopch/byte-of-python,Remove Romanian translation (abandoned)\n\nThe...,diff --git a/translations.md b/translations.md...
1,1,swaroopch/byte-of-python,Add link to Real Python on editor,diff --git a/first_steps.md b/first_steps.md\n...
2,2,swaroopch/byte-of-python,Fix typo,diff --git a/README.md b/README.md\nindex 85a9...


## Global variables

In [11]:
# Model types and sizes
MODEL_SIZES = ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]
MODEL_SIZE = MODEL_SIZES[0]
MODEL_TYPES = [TFT5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoModelForCausalLM ]
MODEL_TYPE = MODEL_TYPES[1]

# Data
FEATURES = "Diff"
TARGET = "Message"
DATA = data1

# Determine task for teh model (summarize, translate)
PREFIX = "summarize: "

# Determine maximum input for features and the target
MAX_INPUT_LENGTH = 540
MAX_TARGET_LENGTH = 540

# Fine-Tuning Parameters
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
ADAM_BETA1 = 0.9
WEIGHT_DECAY = 0.01

# Testing
GIT_DIFF = "random diff that is going to be used to generate comment when committed"

## Clean DATA

In [12]:
DATA[FEATURES] = DATA[FEATURES].apply(lambda x : str(x))

In [13]:
DATA[TARGET] = DATA[TARGET].apply(lambda x : str(x))

In [14]:
DATA[TARGET] = DATA[TARGET].apply(lambda x : x.replace('\n',' ').replace('.','').replace(',','').replace('(','').replace(')','').replace('!','').replace('+','').replace(':','').replace('/',''))

## Transform data to dataset

In [15]:
ds = datasets.Dataset.from_pandas(DATA)

In [16]:
ds = ds.remove_columns(['Unnamed: 0', 'Repository'])

In [17]:
ds

Dataset({
    features: ['Message', 'Diff'],
    num_rows: 540
})

## Preprocessing

### Manual Tokenizing

In [18]:
# create the tokenizer
tokenizer = T5Tokenizer.from_pretrained(MODEL_SIZE)
tokenizer

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5Tokenizer(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_4

In [19]:
inputs = [PREFIX + difference for difference in ds[FEATURES]]

In [20]:
model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding=True)

In [21]:
labels = tokenizer(text_target=ds[TARGET], max_length=MAX_TARGET_LENGTH, truncation=True, padding=True)

In [22]:
model_inputs["labels"] = labels["input_ids"]

In [23]:
type(model_inputs)

transformers.tokenization_utils_base.BatchEncoding

In [24]:
model_inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [25]:
ds1 = datasets.Dataset.from_dict(model_inputs)

In [26]:
ds1

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 540
})

In [27]:
tts_ds = ds1.train_test_split(test_size=0.3)


In [28]:
train_data = tts_ds["train"]
validation_data = tts_ds["test"]

In [30]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=c5a912232d8e15eb624a64fc047a6da8c1885ea85f6fc8f349a764b0ca2992cd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [31]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [32]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [ ]:
def preprocess_function(df):
    # Setup the tokenizer for features
    inputs = [PREFIX + difference for difference in df[FEATURES]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=df[TARGET], max_length=MAX_TARGET_LENGTH, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


## Instantiate T5 model

In [33]:
# create the model
model = MODEL_TYPE.from_pretrained(MODEL_SIZE)
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [36]:
# Create new training model based on T5 model and given new parameters
def create_training_model(model, tokenizer, train_data, validation_data):

    # create arguments for training the model
    args = Seq2SeqTrainingArguments(
        f"{MODEL_SIZE}-finetuned-xsum",
        evaluation_strategy = "epoch",
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size= BATCH_SIZE,
        per_device_eval_batch_size= BATCH_SIZE,
        weight_decay=WEIGHT_DECAY,
        adam_beta1 = ADAM_BETA1,
        save_total_limit=3,
        num_train_epochs=1,
        predict_with_generate=True,
        fp16=True,
        push_to_hub=False)

    # pad inputs and labels to the maximum length in the batch
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # model creation
    trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=validation_data,
    data_collator=data_collator,
    tokenizer=tokenizer)
    #compute_metrics=compute_metrics

    # return new training model
    return trainer

In [37]:
#Commit to Excellence model
CTE_model = create_training_model(model, tokenizer, train_data, validation_data)

In [ ]:
# train CTE_model
CTE_model.train()

Epoch,Training Loss,Validation Loss
1,No log,8.354711


TrainOutput(global_step=24, training_loss=9.761449813842773, metrics={'train_runtime': 20.8397, 'train_samples_per_second': 18.138, 'train_steps_per_second': 1.152, 'total_flos': 53956969758720.0, 'train_loss': 9.761449813842773, 'epoch': 1.0})

In [38]:
encoding_test = tokenizer.encode((PREFIX + GIT_DIFF),return_tensors="tf")

In [39]:
model_inputs1 = tokenizer(GIT_DIFF, max_length=MAX_INPUT_LENGTH, truncation=True, padding=True)

In [40]:
type(model_inputs1)

transformers.tokenization_utils_base.BatchEncoding

In [41]:
model_inputs1.keys()

dict_keys(['input_ids', 'attention_mask'])

In [67]:
model_inputs1

{'input_ids': [6504, 20624, 24, 19, 352, 12, 36, 261, 12, 3806, 1670, 116, 3454, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
new_df = pd.DataFrame()
new_df["input_ids"] = model_inputs1["input_ids"]
new_df["attention_mask"] = model_inputs1["attention_mask"]


In [68]:
tokenizer.decode(model_inputs1['input_ids'])

'random diff that is going to be used to generate comment when committed</s>'

In [66]:
CTE_model

AttributeError: ignored

In [55]:
CTE_model.predict(model_inputs1)

KeyError: ignored

In [69]:
CTE_model.save_model('cte_model_230830-t5-small')

In [70]:
!ls -a

.   .config		       data2.csv    t5-small-finetuned-xsum
..  cte_model_230830-t5-small  sample_data


In [ ]:
dir(CTE_model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_sm_patterns_to_gitignore',
 '_created_lr_scheduler',
 '_finish_current_push',
 '_gather_and_numpify',
 '_gen_kwargs',
 '_get_collator_with_removed_columns',
 '_get_eval_sampler',
 '_get_learning_rate',
 '_get_output_dir',
 '_get_train_sampler',
 '_globalstep_last_logged',
 '_hp_search_setup',
 '_inner_training_loop',
 '_issue_warnings_after_load',
 '_load_best_model',
 '_load_from_checkpoint',
 '_load_optimizer_and_scheduler',
 '_load_rng_state',
 '_loggers_initialized',
 '_maybe_log_save_evaluate',
 '_memory_tracker',
 '_move_model_to_device',
 '_nested_gather',
 '_pad_tensors_to_max_len',
 '_prepare_input',
 '_pre

In [ ]:
#Testing

def generate_comment(tokenizer, git_diff):
    preprocessed_text= PREFIX + git_diff

    encoding_test = tokenizer.encode(preprocessed_text,return_tensors="tf")

    summary_ids = CTE_model.generate(encoding_test, min_length=60, max_length=80)

    commit_message = tokenizer.decode(summary_ids[0])

    return commit_message

In [ ]:
generate_comment(tokenizer, GIT_DIFF)


AttributeError: ignored